# Train and test the seq2seq model

In [1]:
!pip install tensorflow==1.12

     |████████████████████████████████| 83.1MB 78kB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 3.1MB 38.9MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
import json
import os
import time


import tensorflow as tf

import tensorflow.contrib.legacy_seq2seq as seq2seq

from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!python3 "/content/drive/My Drive/Colab Notebooks/download.py"

Downloading:  train_images.npy
tcmalloc: large alloc 1414594560 bytes == 0x67e3e000 @  0x7fd0942011e7 0x59221c 0x598e86 0x50a51c 0x50c1f4 0x507f24 0x588ddb 0x4b6271 0x55de6d 0x50c11b 0x507f24 0x509202 0x594b01 0x59fe1e 0x50d596 0x507f24 0x509202 0x594b01 0x59fe1e 0x50d596 0x507f24 0x588fac 0x59fe1e 0x50d596 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x50b053 0x634dd2
Downloading:  train_labels.npy
Downloading:  test_images.npy
Downloading:  test_labels.npy
Downloading:  iseq_n.npy
Downloading:  oseq_n.npy
Downloading:  normalized.zip
Downloading:  formulas.zip
Extracting zip file normalized.zip
Extracting zip file formulas.zip


In [4]:
xo = np.load("iseq_n.npy")
yo = np.load("oseq_n.npy")

FileNotFoundError: ignored

In [ ]:
x = xo[:,:26,:]
y = yo[:,:26]

In [ ]:
nxchars = x.shape[2]
print(nxchars)
ltokens = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', '#leq', '#neq', '#geq', '#alpha',
                            '#beta', '#lambda', '#lt', '#gt', 'x', 'y', '^', '#frac', '{', '}' ,' ']
print("#ltokens: ", len(ltokens))

30
#ltokens:  28


In [ ]:
x_seq_length = len(x[0])
y_seq_length = len(y[0])- 1
print(x_seq_length, y_seq_length)

26 25


In [ ]:
def batch_data(x, y, batch_size):
    shuffle = np.random.permutation(len(x))
    start = 0
#     from IPython.core.debugger import Tracer; Tracer()()
    x = x[shuffle]
    y = y[shuffle]
    while start + batch_size <= len(x):
        yield x[start:start+batch_size], y[start:start+batch_size]
        start += batch_size

In [ ]:
batch_size = 512
nodes = 256
embed_size = 20

# Tensor where we will feed the data into graph
inputs = tf.placeholder(tf.float32, (None, x_seq_length, nxchars), 'inputs')
outputs = tf.placeholder(tf.int32, (None, None), 'output')
targets = tf.placeholder(tf.int32, (None, None), 'targets')

# Embedding layers
output_embedding = tf.Variable(tf.random_uniform((len(ltokens)+1, embed_size), -1.0, 1.0), name='dec_embedding')
date_output_embed = tf.nn.embedding_lookup(output_embedding, outputs)

with tf.variable_scope("encoding") as encoding_scope:
    lstm_enc = tf.contrib.rnn.BasicLSTMCell(nodes)
    _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=inputs, dtype=tf.float32)

with tf.variable_scope("decoding") as decoding_scope:
    lstm_dec = tf.contrib.rnn.BasicLSTMCell(nodes)
    dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec, inputs=date_output_embed, initial_state=last_state)
#connect outputs to 
logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=len(ltokens)+1, activation_fn=None) 
with tf.name_scope("optimization"):
    # Loss function
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, y_seq_length]))
    # Optimizer
    optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


Train the graph above:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
print(y_train[0])

[28  2  8  3 18 23  2 11  3  0  7 27 13 27  6  6  8 27 27 27 27 27 27 27
 27 27]


In [ ]:
def save(sess):
    saver = tf.train.Saver(None)

    save_path = saver.save(sess, save_path="seq_mod/model", global_step=None)
    print('model saved at %s' % save_path)

In [ ]:
def restore(sess):
    saver = tf.train.Saver(None)
    path = "seq_mod/model"
    saver.restore(sess, save_path=path)
    print('model restored from %s' % path)

In [ ]:
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    restore(sess)
    epochs = 1
    for epoch_i in range(epochs):
        start_time = time.time()
        for batch_i, (source_batch, target_batch) in enumerate(batch_data(X_train, y_train, batch_size)):
            _, batch_loss, batch_logits = sess.run([optimizer, loss, logits],
                feed_dict = {inputs: source_batch,
                 outputs: target_batch[:, :-1],
                 targets: target_batch[:, 1:]})

        accuracy = np.mean(batch_logits.argmax(axis=-1) == target_batch[:,1:])
        print('Epoch {:3} Loss: {:>6.3f} Accuracy: {:>6.4f} Epoch duration: {:>6.3f}s'.format(epoch_i, batch_loss, 
                                                                          accuracy, time.time() - start_time))

        source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

        dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
        for i in range(y_seq_length):
            batch_logits = sess.run(logits,
                        feed_dict = {inputs: source_batch,
                         outputs: dec_input})
            prediction = batch_logits[:,-1].argmax(axis=-1)
            dec_input = np.hstack([dec_input, prediction[:,None]])

        print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))     
        if epoch_i % 5 == 0:
            save(sess)
    
    save(sess)

INFO:tensorflow:Restoring parameters from seq_mod/model
model restored from seq_mod/model
Epoch   0 Loss:  0.002 Accuracy: 0.9998 Epoch duration: 154.970s
Accuracy on test set is:  0.948
model saved at seq_mod/model
model saved at seq_mod/model


Translate on test set

In [ ]:
with tf.Session() as sess: 
    restore(sess)
    batch_size = 512
    source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

    dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
    for i in range(y_seq_length):
        batch_logits = sess.run(logits,
                    feed_dict = {inputs: source_batch,
                     outputs: dec_input})
        prediction = batch_logits[:,-1].argmax(axis=-1)
        dec_input = np.hstack([dec_input, prediction[:,None]])

    print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))

INFO:tensorflow:Restoring parameters from seq_mod/model
model restored from seq_mod/model
Accuracy on test set is:  0.951


Let's randomly take two from this test set and see what it spits out:

In [ ]:
i = 102
print(dec_input[i,:])
print(ltokens)
seq = ""
for c in dec_input[i,1:]:
    c = int(c)
    if c != 28:
        seq += ltokens[c] 
        
print(seq)

seq = ""
for c in target_batch[i,1:]:
    c = int(c)
    
    if c != 28:
        seq += ltokens[c] 

print("Correct: ", seq)

[28.  6.  1.  8. 17. 23.  3. 11.  4.  7.  5. 27. 13. 27.  1.  1.  2.  2.
 27. 27. 27. 27. 27. 27. 27. 27.]
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', '#leq', '#neq', '#geq', '#alpha', '#beta', '#lambda', '#lt', '#gt', 'x', 'y', '^', '#frac', '{', '}', ' ']
618#beta^3+475 #leq 1122        
Correct:  618#beta^3+475 #leq 1122        


In [ ]:
with tf.Session() as sess: 
    restore(sess)
    batch_size = 512
    source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

    dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
    for i in range(y_seq_length):
        batch_logits = sess.run(logits,
                    feed_dict = {inputs: source_batch,
                     outputs: dec_input})
        prediction = batch_logits[:,-1].argmax(axis=-1)
        dec_input = np.hstack([dec_input, prediction[:,None]])

    print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))
    cc = 0
    fcc = 0
    fc = 0
    fl = 0
    nl = 0
    for i in range(len(dec_input)):
        feq = False
        pseq = ""
        for c in dec_input[i,1:]:
            c = int(c)
            if c != 28:
                pseq += ltokens[c] 

        cseq = ""
        cseql = ""
        for c in target_batch[i,1:]:
            c = int(c)

            if c != 28:
                cseq += ltokens[c] 
                cseql += ltokens[c][0]
                if ltokens[c] == "#frac":
                    fc += 1
                    feq = True

        if pseq == cseq:
            cc += 1
            if feq:
                fcc += 1
        cseql = cseql.rstrip()
        if feq:
            fl += len(cseql)
        else:
            nl += len(cseql)
        
    print("Accuracy %.2f %%" % (cc/len(dec_input)*100))  
    print("Accuracy for fraction equations %.2f %%" % (fcc/fc*100))
    print("Accuracy for simple equations %.2f %%" % ((cc-fcc)/(len(dec_input)-fc)*100))
    print("Average length frac: %.1f" % (fl/fc))
    print("Average length simple: %.1f" % (nl/(len(dec_input)-fc)))

INFO:tensorflow:Restoring parameters from seq_mod/model
model restored from seq_mod/model
Accuracy on test set is:  0.955
Accuracy 64.26 %
Accuracy for fraction equations 45.61 %
Accuracy for simple equations 80.59 %
Average length frac: 23.4
Average length simple: 15.6


In [ ]:
from skimage import io
import matplotlib.pyplot as plt
from Latex.Latex import Latex

with tf.Session() as sess: 
  restore(sess)

  formula = io.imread("1+2+3 #g 0.jpg")
  plt.figure(figsize=(20,10))
  plt.imshow(formula, cmap="gray")
  plt.show()
  


ModuleNotFoundError: ignored